<a href="https://colab.research.google.com/github/majambo/rangeland/blob/main/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap
geemap.update_package()

In [2]:
import os
import ee
import geemap

In [ ]:
Map = geemap.Map(center=(4.18, 38.57), zoom=8)
Map

In [7]:
import numpy as np
import pandas as pd

In [8]:
# manage the date formating as per your requirements
# Mine is in format of YYYYMMdd
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [90]:
sm = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY_AGGR") \
              .filter(ee.Filter.date('2020-01-01', '2023-09-30')) \
              .select('volumetric_soil_water_layer_1')

In [67]:
ndvi = ee.ImageCollection("MODIS/061/MOD13A2") \
            .filter(ee.Filter.date('2020-01-01', '2023-09-30')) \
            .select('EVI')

In [68]:

rainfall = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD') \
              .filter(ee.Filter.date('2020-01-01', '2023-09-30'))

In [31]:
Map = geemap.Map(center=(4.18, 38.57), zoom=8)
Map

Map(center=[4.18, 38.57], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

In [44]:
Map.addLayer(ndvi, {}, 'Average Evapotranspiration');

In [79]:
Map.addLayer(ee_fc, {}, "Borana")

In [53]:
plot_df = pd.read_excel('gridpoints.xlsx')
plot_df

,Lon,Lat
0,36.7,5.3
1,36.8,5.3
2,36.9,5.3
3,37.0,5.3
4,37.1,5.3
...,...,...
584,39.3,3.5
585,39.4,3.5
586,39.5,3.5
587,39.6,3.5


In [ ]:
features=[]
for index, row in plot_df.iterrows():
#     print(dict(row))
#     construct the geometry from dataframe
    poi_geometry = ee.Geometry.Point([row['Lon'], row['Lat']])
#     print(poi_geometry)
#     construct the attributes (properties) for each point
    poi_properties = dict(row)
#     construct feature combining geometry and properties
    poi_feature = ee.Feature(poi_geometry, poi_properties)
#     print(poi_feature)
    features.append(poi_feature)

# final Feature collection assembly
ee_fc = ee.FeatureCollection(features)
ee_fc.getInfo()

In [60]:
def rasterExtraction(image):
    feature = image.sampleRegions(
        collection = ee_fc, # feature collection here
        scale = 10000 # Cell size of raster
    )
    return feature

In [91]:
sm1 = sm.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()
ndvi1 = ndvi.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()
rainfall1 = rainfall.filterBounds(ee_fc).map(addDate).map(rasterExtraction).flatten()

In [92]:
sample_result = sm1.first().getInfo()
sample_result

{'type': 'Feature',
 'geometry': None,
 'id': '202001_0_0',
 'properties': {'Lat': 5.3,
  'Lon': 36.7,
  'date': 20200101,
  'volumetric_soil_water_layer_1': 0.2723448968702747}}

In [75]:
# extract the properties column from feature collection
# column order may not be as our sample data order
columns = list(sample_result['properties'].keys())
print(columns)

['Lat', 'Lon', 'SoilMoi00_10cm_tavg', 'date']


In [ ]:
nested_list = sm1.reduceColumns(ee.Reducer.toList(len(column_df)), column_df).values().get(0)
data = nested_list.getInfo()

# dont forget we need to call the callback method "getInfo" to retrieve the data
df = pd.DataFrame(data, columns=column_df)
# we obtain the data frame as per our demand
df

In [93]:
url_csv = sm1.getDownloadURL('csv')
# click the link below, this will download CSV directly to your local device
# You can use this url and download with python request module, I will leave that to you
url_csv

'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/tables/6a16625f5bbd8c529f7c3a56e69e5d70-322f28e358000713830e28886152823b:getFeatures'